# Lecture de l'alphabet des signes sur un flux vidéo

<hr>

In [1]:
import skimage
import numpy as np

import matplotlib.pyplot as plt

import tensorflow as tf

import warnings
warnings.filterwarnings('ignore')

#### Chargement des modèles

In [ ]:
# modèle de détection de la main, entraîné avec Tensorflow Detection API
model_loc = tf.saved_model.load('XXXXXXXXloc') 
# la fonction qui sert à la prédiction se trouve dans le module "signature" du modèle entraîné avec Tensorflow
model_locX = model_loc.signatures['serving_default']

# modèle de classification du signe, entraîné avec Keras
model_rec = tf.keras.models.load_model('XXXXXXXXrec')

#### Création de la fonction de détection et classification des mains/signes sur une image

In [ ]:
def detect_and_classify_handsign(image):
    
    # chargement et prétraitement de l'image :
    input_tensor = tf.convert_to_tensor(image) # conversion de l'image en tenseur - shape=(X, X, 3)
    input_tensor = input_tensor[tf.newaxis,...] # ajout d'un axe pour le batch - shape=(1, X, X, 3)
    
    # on prédit la localisation de la main sur l'image :
    output_dict = model_locX(input_tensor) 
    
    # il va falloir trier ces détections pour ne garder que celles qui correspondent réellement à une main :
    ## 'num_detection' est une clé du dictionnaire output_dict qui donne le nombre de "mains" 
    ## détectées sur l'image.
    num_detections = int(output_dict.pop('num_detections'))
    output_dict = {key:value[0, :num_detections].numpy() for key,value in output_dict.items()}
    ## 'detection_boxes' est une clé du dictionnaire output_dict qui donne les coordonnées des  
    ## rectangles d'encadrement des mains détectées.
    boxes = output_dict['detection_boxes']
    ## 'detection_scores' est une clé du dictionnaire output_dict qui donne la probabilité de 
    ## présence effective de la main dans le rectangle d'encadrement.
    scores = np.array(output_dict['detection_scores'])
    
    # On trie les rectangles d'encadrement pour ne conserver que celui qui détecte une main 
    # avec la plus haute probabilité :
    real_boxe = boxes[np.argmax(scores)]
    
    # on extrait ensuite l'image de la main à partir de l'image d'origine et des coordonnées 
    # du rectangle d'encadrement :
    ## récupération des coordonnées des 4 coins du rectangle d'encadrement sur l'image d'origine.
    x1 = int(image.shape[1]*real_boxe[1])
    x2 = int(image.shape[1]*real_boxe[3])
    y1 = int(image.shape[0]*real_boxe[0])
    y2 = int(image.shape[0]*real_boxe[2])
    
    ## extraction de l'image de la main et prétraîtement en vue du traîtement par le modèle de 
    ## classification (rognage, conversion en niveaux de gris, standartisation, modification de forme).
    cropped = image[y1:y2, x1:x2]
    gray = cv2.cvtColor(cropped, cv2.COLOR_BGR2GRAY) 
    localized = (cv2.resize(gray,(50,50)).astype('float') / 255).reshape(1,50,50,1)
        
    # reconaissance du signe grace au modèle de classification :
    recognized = np.argmax(model_rec.predict(localized)) 
    image = cv2.rectangle(image, (x1, y1), (x2, y2), (255, 255, 0), 1) # on trace le rectangle
    image = cv2.putText(image, LABELS[recognized], (10,300), cv2.FONT_HERSHEY_SIMPLEX, 2, (255,255,0), 2)
    
    return image

#### Création de la fonction de capture et de traitement de l'image Webcam

In [ ]:
def process_video(record_name=None):
    cap = cv2.VideoCapture(0)

    if (cap.isOpened()== False): 
        print("Error opening video stream")
    else :
        # on récupère la hauteur et la largeur de l'image grâce aux méthodes cv2
        global_width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
        global_height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))

    if record_name: # enregistement si un nom de fichier a été spécifié
        result = cv2.VideoWriter(record_name,  cv2.VideoWriter_fourcc(*'MJPG'), 15, (global_width, global_height)) 
        
    while(cap.isOpened()):
        ret, frame = cap.read()
        if ret == True:
            # on appelle la fonction créér ci-dessus pour traîter l'image video
            frame_processed = detect_and_classify_handsign(frame)
            # on retourne l'image retraitée
            cv2.imshow('Frame',frame_processed)

            if cv2.waitKey(1) == 27:
                break

        else: 
            break

    cap.release()
    if record_name:
        result.release()
    cv2.destroyAllWindows()

In [5]:
import cv2
cap = cv2.VideoCapture(0)


In [6]:
cap.release()